In [116]:
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.utils import shuffle

In [117]:
def data_split(x,y):
  xn_train = x[:,:-1]
  xn_label = x[:,-1]
  xn_test = y[:,:-1]
  yn_label = y[:,-1]
  return xn_train, xn_label, xn_test, yn_label

In [143]:
def ref(x,y):
  z = np.zeros([y.shape[0],1])
  for i in range(y.shape[0]):
    if y[i] == 1:
      z[i] = 1
    else:
      z[i] = -1
  #xn = np.column_stack((np.ones([x.shape[0],1],dtype=float),x))
  g = np.multiply(x, z)
  return x, g 

def crit(w,x):
  j = 0
  l = int(x.shape[0])
  for i in range(l):
    if (np.dot(w.T, x[i,:]) <= 0):
      j = (j - np.dot(w.T, x[i,:]))
  return j

def predict(w,x):
  w_pred = np.zeros([int(x.shape[0]),1])
  for i in range(int(x.shape[0])):
    z = np.dot(w.T, x[i,:])
    if z < 0:
      w_pred[i] = 2
    else:
      w_pred[i] = 1
  
  return w_pred

def error(x,y):
  count = 0
  for i in range(int(x.shape[0])):
    if x[i]!=y[i]:
      count = count + 1
  err = (count/int(x.shape[0]))*100
  return err

In [159]:
data_train = np.array(np.loadtxt("/content/dataset1_train.csv", delimiter = ",", dtype = float))
data_test = np.array(np.loadtxt("/content/dataset1_test.csv", delimiter = ",", dtype = float))

In [160]:
xn_train,xn_label,xn_test,y_label = data_split(data_train, data_test)

In [161]:
#For Training Dataset
xn1 = np.column_stack((np.ones([xn_train.shape[0],1],dtype=float),xn_train))
n_features = xn1.shape[1]
X2 = np.zeros((xn1.shape[0], n_features * (n_features + 1) // 2))
k = 0
for i in range(n_features):
    X2[:, k] = xn1[:, i] ** 2      # quadratic term
    k += 1
    for j in range(i + 1, n_features):
        X2[:, k] = xn1[:, i] * xn1[:, j]  # interaction term
        k += 1

# Combine the original and new features
x_new = np.hstack((xn1, X2))
xn, g = ref(x_new,xn_label)

In [154]:
g.shape

(100, 9)

In [177]:
#Testing Dataset
test_xn = np.column_stack((np.ones([xn_test.shape[0],1],dtype=float),xn_test))
n_features = test_xn.shape[1]
X2 = np.zeros((test_xn.shape[0], n_features * (n_features + 1) // 2))
k = 0
for i in range(n_features):
    X2[:, k] = test_xn[:, i] ** 2      # quadratic term
    k += 1
    for j in range(i + 1, n_features):
        X2[:, k] = test_xn[:, i] * test_xn[:, j]  # interaction term
        k += 1

# Combine the original and new features
y_new = np.hstack((test_xn, X2))

In [156]:
xn.shape

(100, 9)

In [157]:
w = np.ones([9,1])
w_out = np.ones([9,10000])

#Training 
l = int(g.shape[0])
epochs = int(10000/l)
axis = []



for i in range(epochs):
  count = 0
  xn,g,xn_label = shuffle(xn,g,xn_label)
  # g = shuffle(g)
  # xn_label = shuffle(xn_label)

  for j in range(l):
    ind = i*epochs + j
    axis.append(ind + 1)
    z = np.dot(w.T, g[j,:])

    if z <= 0:
      w = w + 1*g[j,:].reshape([9,1])
      count = 0
    
    else:
      count = count + 1
    
    w_out[:,ind] = w.T
  
  if count == 100:
    print('Linearly Separable')
    break

J_d = np.zeros(10000)

for j in range(10000):
  J_d[j] = crit(w_out[:,j], g)

w_opt = w_out[:, np.argmin(J_d)]
print('The optimal weights are:', w_opt)

train_label = predict(w_opt,xn)
print('The training classification error is:', error(train_label, xn_label))

#test_xn = np.column_stack((np.ones([xn_test.shape[0],1],dtype=float),xn_test))
test_label = predict(w_opt,y_new)

print('The testing classification error is:', error(test_label, y_label))

Linearly Separable
The optimal weights are: [ -2.         -18.03913097  16.08131885  -2.         -18.03913097
  16.08131885  -8.73095567   1.69430432   6.5621043 ]
The training classification error is: 0.0
The testing classification error is: 0.0


In [96]:
train_label.shape

(100, 1)

In [178]:
w = np.ones([9,1])
w_out = np.ones([9,10000])

#Training 
l = int(g.shape[0])
epochs = int(10000/l)
axis = []
accu_train = []
accu_test = []
k = 0
w_t = []

while (k!=10):
  for i in range(epochs):
    count = 0
    xn,g,xn_label = shuffle(xn,g,xn_label)
    # g = shuffle(g)
    # xn_label = shuffle(xn_label)

    for j in range(l):
      ind = i*epochs + j
      axis.append(ind + 1)
      z = np.dot(w.T, g[j,:])

      if z <= 0:
        w = w + 1*g[j,:].reshape([9,1])
        count = 0
      
      else:
        count = count + 1
      
      w_out[:,ind] = w.T
    
    if count == 100:
      #print('Linearly Separable')
      break

  J_d = np.zeros(10000)

  for j in range(10000):
    J_d[j] = crit(w_out[:,j], g)

  w_opt = w_out[:, np.argmin(J_d)]
  #print('The optimal weights are:', w_opt)
  w_t.append(w_opt)

  train_label = predict(w_opt,xn)
  err = error(train_label, xn_label)
  accu_train.append(100 - err)
  #print('The training classification error is:', error(train_label, xn_label))

  #test_xn = np.column_stack((np.ones([xn_test.shape[0],1],dtype=float),xn_test))
  test_label = predict(w_opt,y_new)
  err_test = error(test_label, y_label)
  accu_test.append(100 - err_test)
  k = k + 1
  #print('The testing classification error is:', error(test_label, y_label))

In [164]:
mean_train = np.mean(np.array(accu_train), axis = 0)
std_train = np.std(np.array(accu_train), axis = 0)
mean_test = np.mean(np.array(accu_test), axis = 0)
std_test = np.std(np.array(accu_test), axis = 0)

print("The mean and standard deviation for training accuracy is as follows: {:.2f}%, {:.2f}".format(mean_train,std_train))
print("The mean and standard deviation for testing accuracy is as follows: {:.2f}%, {:.2f}".format(mean_test,std_test))

The mean and standard deviation for training accuracy is as follows: 100.00%, 0.00
The mean and standard deviation for testing accuracy is as follows: 100.00%, 0.00


In [173]:
data_train = np.array(np.loadtxt("/content/dataset2_train.csv", delimiter = ",", dtype = float))
data_test = np.array(np.loadtxt("/content/dataset2_test.csv", delimiter = ",", dtype = float))

xn_train,xn_label,xn_test,y_label = data_split(data_train, data_test)

#For Training Dataset
xn1 = np.column_stack((np.ones([xn_train.shape[0],1],dtype=float),xn_train))
n_features = xn1.shape[1]
X2 = np.zeros((xn1.shape[0], n_features * (n_features + 1) // 2))
k = 0
for i in range(n_features):
    X2[:, k] = xn1[:, i] ** 2      # quadratic term
    k += 1
    for j in range(i + 1, n_features):
        X2[:, k] = xn1[:, i] * xn1[:, j]  # interaction term
        k += 1

# Combine the original and new features
x_new = np.hstack((xn1, X2))
xn, g = ref(x_new,xn_label)

#Testing Dataset
test_xn = np.column_stack((np.ones([xn_test.shape[0],1],dtype=float),xn_test))
n_features = test_xn.shape[1]
X2 = np.zeros((test_xn.shape[0], n_features * (n_features + 1) // 2))
k = 0
for i in range(n_features):
    X2[:, k] = test_xn[:, i] ** 2      # quadratic term
    k += 1
    for j in range(i + 1, n_features):
        X2[:, k] = test_xn[:, i] * test_xn[:, j]  # interaction term
        k += 1

# Combine the original and new features
y_new = np.hstack((test_xn, X2))

In [166]:
w = np.ones([9,1])
w_out = np.ones([9,10000])

#Training 
l = int(g.shape[0])
epochs = int(10000/l)
axis = []



for i in range(epochs):
  count = 0
  xn,g,xn_label = shuffle(xn,g,xn_label)
  # g = shuffle(g)
  # xn_label = shuffle(xn_label)

  for j in range(l):
    ind = i*epochs + j
    axis.append(ind + 1)
    z = np.dot(w.T, g[j,:])

    if z <= 0:
      w = w + 1*g[j,:].reshape([9,1])
      count = 0
    
    else:
      count = count + 1
    
    w_out[:,ind] = w.T
  
  if count == 100:
    print('Linearly Separable')
    break

J_d = np.zeros(10000)

for j in range(10000):
  J_d[j] = crit(w_out[:,j], g)

w_opt = w_out[:, np.argmin(J_d)]
print('The optimal weights are:', w_opt)

train_label = predict(w_opt,xn)
print('The training classification error is:', error(train_label, xn_label))

#test_xn = np.column_stack((np.ones([xn_test.shape[0],1],dtype=float),xn_test))
test_label = predict(w_opt,y_new)

print('The testing classification error is:', error(test_label, y_label))

Linearly Separable
The optimal weights are: [-4.         -3.01364703  6.4185535  -4.         -3.01364703  6.4185535
  1.7201959  -0.39269427  5.50534088]
The training classification error is: 0.0
The testing classification error is: 5.0


In [174]:
w = np.ones([9,1])
w_out = np.ones([9,10000])

#Training 
l = int(g.shape[0])
epochs = int(10000/l)
axis = []
accu_train = []
accu_test = []
k = 0
w_t = []

while (k!=10):
  for i in range(epochs):
    count = 0
    xn,g,xn_label = shuffle(xn,g,xn_label)
    # g = shuffle(g)
    # xn_label = shuffle(xn_label)

    for j in range(l):
      ind = i*epochs + j
      axis.append(ind + 1)
      z = np.dot(w.T, g[j,:])

      if z <= 0:
        w = w + 1*g[j,:].reshape([9,1])
        count = 0
      
      else:
        count = count + 1
      
      w_out[:,ind] = w.T
    
    if count == 100:
      #print('Linearly Separable')
      break

  J_d = np.zeros(10000)

  for j in range(10000):
    J_d[j] = crit(w_out[:,j], g)

  w_opt = w_out[:, np.argmin(J_d)]
  #print('The optimal weights are:', w_opt)
  w_t.append(w_opt)

  train_label = predict(w_opt,xn)
  err = error(train_label, xn_label)
  accu_train.append(100 - err)
  #print('The training classification error is:', error(train_label, xn_label))

  #test_xn = np.column_stack((np.ones([xn_test.shape[0],1],dtype=float),xn_test))
  test_label = predict(w_opt,y_new)
  err_test = error(test_label, y_label)
  accu_test.append(100 - err_test)
  k = k + 1
  #print('The testing classification error is:', error(test_label, y_label))

mean_train = np.mean(np.array(accu_train), axis = 0)
std_train = np.std(np.array(accu_train), axis = 0)
mean_test = np.mean(np.array(accu_test), axis = 0)
std_test = np.std(np.array(accu_test), axis = 0)

print("The mean and standard deviation for training accuracy is as follows: {:.2f}%, {:.2f}".format(mean_train,std_train))
print("The mean and standard deviation for testing accuracy is as follows: {:.2f}%, {:.2f}".format(mean_test,std_test))

The mean and standard deviation for training accuracy is as follows: 100.00%, 0.00
The mean and standard deviation for testing accuracy is as follows: 95.00%, 0.00


In [175]:
data_train = np.array(np.loadtxt("/content/dataset3_train.csv", delimiter = ",", dtype = float))
data_test = np.array(np.loadtxt("/content/dataset3_test.csv", delimiter = ",", dtype = float))

xn_train,xn_label,xn_test,y_label = data_split(data_train, data_test)

#For Training Dataset
xn1 = np.column_stack((np.ones([xn_train.shape[0],1],dtype=float),xn_train))
n_features = xn1.shape[1]
X2 = np.zeros((xn1.shape[0], n_features * (n_features + 1) // 2))
k = 0
for i in range(n_features):
    X2[:, k] = xn1[:, i] ** 2      # quadratic term
    k += 1
    for j in range(i + 1, n_features):
        X2[:, k] = xn1[:, i] * xn1[:, j]  # interaction term
        k += 1

# Combine the original and new features
x_new = np.hstack((xn1, X2))
xn, g = ref(x_new,xn_label)

#Testing Dataset
test_xn = np.column_stack((np.ones([xn_test.shape[0],1],dtype=float),xn_test))
n_features = test_xn.shape[1]
X2 = np.zeros((test_xn.shape[0], n_features * (n_features + 1) // 2))
k = 0
for i in range(n_features):
    X2[:, k] = test_xn[:, i] ** 2      # quadratic term
    k += 1
    for j in range(i + 1, n_features):
        X2[:, k] = test_xn[:, i] * test_xn[:, j]  # interaction term
        k += 1

# Combine the original and new features
y_new = np.hstack((test_xn, X2))

In [171]:
w = np.ones([9,1])
w_out = np.ones([9,10000])

#Training 
l = int(g.shape[0])
epochs = int(10000/l)
axis = []



for i in range(epochs):
  count = 0
  xn,g,xn_label = shuffle(xn,g,xn_label)
  # g = shuffle(g)
  # xn_label = shuffle(xn_label)

  for j in range(l):
    ind = i*epochs + j
    axis.append(ind + 1)
    z = np.dot(w.T, g[j,:])

    if z <= 0:
      w = w + 1*g[j,:].reshape([9,1])
      count = 0
    
    else:
      count = count + 1
    
    w_out[:,ind] = w.T
  
  if count == 100:
    print('Linearly Separable')
    break

J_d = np.zeros(10000)

for j in range(10000):
  J_d[j] = crit(w_out[:,j], g)

w_opt = w_out[:, np.argmin(J_d)]
print('The optimal weights are:', w_opt)

train_label = predict(w_opt,xn)
print('The training classification error is:', error(train_label, xn_label))

#test_xn = np.column_stack((np.ones([xn_test.shape[0],1],dtype=float),xn_test))
test_label = predict(w_opt,y_new)

print('The testing classification error is:', error(test_label, y_label))

The optimal weights are: [  5.         -15.57473462 -11.37383023   5.         -15.57473462
 -11.37383023  18.38388368  12.06365555  10.22463312]
The training classification error is: 7.000000000000001
The testing classification error is: 8.0


In [176]:
w = np.ones([9,1])
w_out = np.ones([9,10000])

#Training 
l = int(g.shape[0])
epochs = int(10000/l)
axis = []
accu_train = []
accu_test = []
k = 0
w_t = []

while (k!=10):
  for i in range(epochs):
    count = 0
    xn,g,xn_label = shuffle(xn,g,xn_label)
    # g = shuffle(g)
    # xn_label = shuffle(xn_label)

    for j in range(l):
      ind = i*epochs + j
      axis.append(ind + 1)
      z = np.dot(w.T, g[j,:])

      if z <= 0:
        w = w + 1*g[j,:].reshape([9,1])
        count = 0
      
      else:
        count = count + 1
      
      w_out[:,ind] = w.T
    
    if count == 100:
      #print('Linearly Separable')
      break

  J_d = np.zeros(10000)

  for j in range(10000):
    J_d[j] = crit(w_out[:,j], g)

  w_opt = w_out[:, np.argmin(J_d)]
  #print('The optimal weights are:', w_opt)
  w_t.append(w_opt)

  train_label = predict(w_opt,xn)
  err = error(train_label, xn_label)
  accu_train.append(100 - err)
  #print('The training classification error is:', error(train_label, xn_label))

  #test_xn = np.column_stack((np.ones([xn_test.shape[0],1],dtype=float),xn_test))
  test_label = predict(w_opt,y_new)
  err_test = error(test_label, y_label)
  accu_test.append(100 - err_test)
  k = k + 1
  #print('The testing classification error is:', error(test_label, y_label))

mean_train = np.mean(np.array(accu_train), axis = 0)
std_train = np.std(np.array(accu_train), axis = 0)
mean_test = np.mean(np.array(accu_test), axis = 0)
std_test = np.std(np.array(accu_test), axis = 0)

print("The mean and standard deviation for training accuracy is as follows: {:.2f}%, {:.2f}".format(mean_train,std_train))
print("The mean and standard deviation for testing accuracy is as follows: {:.2f}%, {:.2f}".format(mean_test,std_test))

The mean and standard deviation for training accuracy is as follows: 95.60%, 1.50
The mean and standard deviation for testing accuracy is as follows: 91.80%, 0.40
